<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_PyTorch/blob/main/BidirectionalRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import warnings
warnings.filterwarnings('ignore')
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences

features = 2000  # Number of most frequent words to consider
max_len = 50     # Maximum length of each sequence

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=features)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [12]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.LongTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.LongTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Create TensorDataset objects
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoader objects
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
import torch.nn as nn

class PyTorchBiRNN(nn.Module):
    def __init__(self, features, embedding_dim, hidden_units, max_len):
        super().__init__()
        # Embedding layer
        self.embedding = nn.Embedding(num_embeddings=features, embedding_dim=embedding_dim)

        # Bidirectional SimpleRNN layer
        # Note: torch.nn.RNN corresponds to Keras SimpleRNN, not LSTM or GRU
        self.rnn = nn.RNN(input_size=embedding_dim,
                          hidden_size=hidden_units,
                          batch_first=True,
                          bidirectional=True)

        # Dense (Linear) layer
        # Bidirectional RNN outputs 2 * hidden_units
        self.fc = nn.Linear(in_features=2 * hidden_units, out_features=1)

        # Sigmoid activation for binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Pass input through embedding layer
        embedded = self.embedding(x)

        # Pass embedded sequence through RNN layer
        # output: tensor containing the output features (h_t) from the last layer of the RNN
        # h_n: tensor containing the final hidden state for each element in the batch
        output, h_n = self.rnn(embedded)

        # For a single-layer bidirectional RNN, h_n will have shape (num_directions * num_layers, batch, hidden_size)
        # For bidirectional=True, num_directions=2. So h_n shape will be (2, batch, hidden_size).
        # h_n[-2, :, :] is the last hidden state of the forward RNN
        # h_n[-1, :, :] is the last hidden state of the backward RNN
        # Concatenate them to get the combined hidden state
        final_hidden_state = torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=1)

        # Pass the combined hidden state through the linear layer
        linear_out = self.fc(final_hidden_state)

        # Apply sigmoid activation
        return self.sigmoid(linear_out)

In [14]:
embedding_dim = 128
hidden_units = 64

model = PyTorchBiRNN(features, embedding_dim, hidden_units, max_len)

criterion = nn.BCEWithLogitsLoss()

learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


PyTorchRNN(
  (embedding): Embedding(2000, 128)
  (rnn): RNN(128, 64, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [15]:
epochs = 5
train_losses = []

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        # Ensure y_batch is the correct shape, criterion expects (batch_size, 1) for BCEWithLogitsLoss
        loss = criterion(outputs, y_batch.unsqueeze(1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    train_losses.append(average_loss)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {average_loss:.4f}")

print("Training complete.")

Epoch 1/5, Loss: 0.6827
Epoch 2/5, Loss: 0.6860
Epoch 3/5, Loss: 0.6651
Epoch 4/5, Loss: 0.6670
Epoch 5/5, Loss: 0.6689
Training complete.


In [16]:
import numpy as np
from sklearn.metrics import classification_report

total_loss = 0
correct_predictions = 0
total_samples = 0

all_preds = []
all_labels = []

model.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Disable gradient calculations for evaluation
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        outputs = model(X_batch)
        loss = criterion(outputs, y_batch.unsqueeze(1)) # criterion is BCEWithLogitsLoss
        total_loss += loss.item()

        predictions = (torch.sigmoid(outputs) > 0.5).long()

        all_labels.extend(y_batch.cpu().numpy())
        all_preds.extend(predictions.cpu().numpy())

        correct_predictions += (predictions.squeeze() == y_batch).sum().item()
        total_samples += y_batch.size(0)

average_loss = total_loss / len(test_loader)
accuracy = correct_predictions / total_samples

print(f"Test Loss: {average_loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Generate classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=['Negative', 'Positive']))

Test Loss: 0.6712
Test Accuracy: 0.5000

Classification Report:
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00     12500
    Positive       0.50      1.00      0.67     12500

    accuracy                           0.50     25000
   macro avg       0.25      0.50      0.33     25000
weighted avg       0.25      0.50      0.33     25000

